In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os

In [3]:
stats = pd.read_csv(os.path.join("MLB_stats.csv"))
stats.head()

,Team,Season,Date,Vs,Score,Away Starter,Home Starter,BAL Line,O/U,Unnamed: 9,FIELD,Game Result,Runs Scr,Runs Alw,Margin,Side Line,O/U Result,O/U Line,O/U Odds
0,Baltimore,2010.0,10/3/10,DET,L 2-4,Phil Coke (L),Bradley Bergesen (R),L -155,U 8.5 -106,NaN,HOME,L,2.0,4.0,-2.0,-155.0,U,8.5,-106
1,Baltimore,2010.0,10/2/10,DET,W 2-1,Armando Galarraga (R),Brian Matusz (L),W -155,U 8.5 -116,NaN,HOME,W,2.0,1.0,1.0,-155.0,U,8.5,-116
2,Baltimore,2010.0,10/1/10,DET,W 2-1,Rick Porcello (R),Jeremy Guthrie (R),W -145,U 8 -106,NaN,HOME,W,2.0,1.0,1.0,-145.0,U,8.0,-106
3,Baltimore,2010.0,10/1/10,DET,W 10-6,Jeremy Bonderman (R),Chris Tillman (R),W -120,O 9 107,NaN,HOME,W,10.0,6.0,4.0,-120.0,O,9.0,107
4,Baltimore,2010.0,9/29/10,@ TB,W 2-0,Kevin Millwood (R),Jeff Niemann (R),W 176,U 9 -107,NaN,AWAY,W,2.0,0.0,2.0,176.0,U,9.0,-107


In [8]:
# Assign X (data) and y (target)
X = stats.drop("Game Result", axis=1)
y = stats["Game Result"]
print(X.shape, y.shape)

(58074, 18) (58074,)


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [13]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()